# Chat Message Memory

Chat Message Memory specializes in storing conversation history between human and an LLM model. 

In LangChain, chat message memory is composed of two main parts, the memory class itself and the `BaseChatMessageHistory` object it is wrapping on. `BaseChatMessageHistory` object defines how conversation history data is serialized and persisted, such as through a local file system or external database. The memory class controls how the chat message memory is presented to the LLM model in the conversation.

In [2]:
# Code to show initialization and explicitly show which part is which part.

from langchain.memory import ConversationBufferMemory, ChatMessageHistory

chat_history = ChatMessageHistory()

# ConversationBufferMemory is a chat message memory object.
chat_memory = ConversationBufferMemory(
    chat_memory=chat_history # <-- This is the `BaseChatMessageHistory` object
)

If you do not explicitly define a `BaseChatMessageHistory` object in `chat_memory` argument during initialization, chat message memory will use a default message storage option, which is `ChatMessageHistory`.

## Type of chat memory

In this section, we will show some examples of memory classes available in LangChain. You can learn more about various types of `BaseChatMessageHistory` in another notebook [here](./chat_message_history.ipynb).

### ConversationBufferMemory

We first start with `ConversationBufferMemory` which is just a simple wrapper around the underlying `BaseChatMessageHistory` that return the messages stored in it without any modification.

In [3]:
# We create a chat message history that stores some dummy conversation.

from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()
chat_history.add_user_message("Hello!")
chat_history.add_ai_message("Hey there, how may I assist you today?")
chat_history.add_user_message("Answer briefly. What are the first 3 colors of a rainbow?")
chat_history.add_ai_message("The first three colors of a rainbow are red, orange, and yellow.")

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

memory = ConversationBufferMemory(chat_memory=chat_history)
chat_model = ChatOpenAI(temperature=0)

conversation = ConversationChain(
    llm=chat_model,
    memory=memory,
    verbose=True
)

In [5]:
conversation.run("And the next 4?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI: Hey there, how may I assist you today?
Human: Answer briefly. What are the first 3 colors of a rainbow?
AI: The first three colors of a rainbow are red, orange, and yellow.
Human: And the next 4?
AI:

> Finished chain.


'The next four colors of a rainbow are green, blue, indigo, and violet.'

### ConversationBufferWindowMemory

While `ConversationBufferMemory` is the most accurate way to incorporate information about previous conversations when querying an LLM, it is certainly not the most efficient way, especially when the model context length is low. Besides, when it holds conversation data longer than the context length supported by the LLM model, unexpected behaviors may occur during the LLM output generation process.

`ConversationBufferWindowMemory` mitigates the problem by returning only the last K interactions. In other words, it keeps a sliding window of the most recent interactions to minimize the chance the returned messages will exceed the model 

In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, chat_memory=chat_history) # Only incorporate the last 2 interactions
conversation = ConversationChain(
    llm=chat_model,
    memory=memory,
    verbose=True
)

In [7]:
conversation.run("That's interesting? Why do rainbows have so many colors?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Answer briefly. What are the first 3 colors of a rainbow?
AI: The first three colors of a rainbow are red, orange, and yellow.
Human: And the next 4?
AI: The next four colors of a rainbow are green, blue, indigo, and violet.
Human: That's interesting? Why do rainbows have so many colors?
AI:

> Finished chain.


'Rainbows have so many colors because they are caused by the refraction and reflection of sunlight through water droplets in the air. This process separates the different wavelengths of light, creating the spectrum of colors we see in a rainbow.'

### ConversationSummaryMemory

`ConversationSummaryMemory` adopts a different approach. Instead of outputing previous conversation verbatim, it generates a summary of previous conversation. This allows us to incorporate more information from previous interactions, at the expense of detailness.

In [8]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory.from_messages(
    chat_memory=chat_history, 
    llm=ChatOpenAI(temperature=0) # <-- You need an LLM to summarize the conversation.
)
conversation = ConversationChain(
    llm=chat_model,
    memory=memory,
    verbose=True
)

In [9]:
conversation.run("It is so hard to remember the order of the colors!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and asks it to name the first three colors of a rainbow. The AI responds with "red, orange, and yellow" and when asked for the next four, it says "green, blue, indigo, and violet." The human asks why rainbows have so many colors, and the AI explains that it's due to the refraction and reflection of sunlight through water droplets in the air, which separates the different wavelengths of light and creates the spectrum of colors we see in a rainbow.
Human: It is so hard to remember the order of the colors!
AI:

> Finished chain.


'Yes, it can be difficult to remember the order of the colors in a rainbow. One way to remember is to use the acronym ROYGBIV, which stands for red, orange, yellow, green, blue, indigo, and violet.'

These are just a handful of chat message memories available in LangChain. You can go [here]() to see the full list of memory classes.

## Output type

By default, chat message memory classes in LangChain will return string when accessed using `load_memory_variables`. Most memory class (with the exception of `ConversationStringBufferHistory`) exposes the `return_messages` boolean field during initialization, which when set to `True`, return a list of messages objects instead of string. This can be helpful when you are using `MessagePlaceholder` (see [Chat Prompt Template](../../prompts/chat_prompt_template.ipynb) if you are unsure what it is) instead of prompt template.

In [10]:
memory = ConversationBufferWindowMemory(
    chat_memory=chat_history,
    k=3,
    return_messages=True
)

In [11]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='And the next 4?', additional_kwargs={}, example=False),
  AIMessage(content='The next four colors of a rainbow are green, blue, indigo, and violet.', additional_kwargs={}, example=False),
  HumanMessage(content="That's interesting? Why do rainbows have so many colors?", additional_kwargs={}, example=False),
  AIMessage(content='Rainbows have so many colors because they are caused by the refraction and reflection of sunlight through water droplets in the air. This process separates the different wavelengths of light, creating the spectrum of colors we see in a rainbow.', additional_kwargs={}, example=False),
  HumanMessage(content='It is so hard to remember the order of the colors!', additional_kwargs={}, example=False),
  AIMessage(content='Yes, it can be difficult to remember the order of the colors in a rainbow. One way to remember is to use the acronym ROYGBIV, which stands for red, orange, yellow, green, blue, indigo, and violet.', additional_kwar

## String output customization

When formatted as a string, you can customize the name of the entity using `human_prefix` and `ai_prefix`. This setting has no effect when the output is a message object (i.e. when `return_messages` is `True`).

In [12]:
memory = ConversationBufferWindowMemory(
    chat_memory=chat_history,
    k=3,
    human_prefix="Learner",
    ai_prefix="Teacher"
)

In [13]:
print(memory.load_memory_variables({})['history'])

Learner: And the next 4?
Teacher: The next four colors of a rainbow are green, blue, indigo, and violet.
Learner: That's interesting? Why do rainbows have so many colors?
Teacher: Rainbows have so many colors because they are caused by the refraction and reflection of sunlight through water droplets in the air. This process separates the different wavelengths of light, creating the spectrum of colors we see in a rainbow.
Learner: It is so hard to remember the order of the colors!
Teacher: Yes, it can be difficult to remember the order of the colors in a rainbow. One way to remember is to use the acronym ROYGBIV, which stands for red, orange, yellow, green, blue, indigo, and violet.
